In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import utils
import pandas as pd
import random
import cv2
import numpy as np
import warnings
import os
from PIL import Image
from skimage.feature import hog

In [2]:

warnings.filterwarnings(action='ignore', category=UserWarning)

In [3]:
train = pd.read_csv('../fashionmnist/fashion-mnist_train.csv')
augmented_train = train.copy()
X_train = augmented_train.drop(['label'],axis = 1)
X_label = augmented_train['label']

X_train = X_train.astype('float32')

In [4]:
label_dict = {
    "T-shirt/top":0,
    "Trouser":1,
    "Pullover":2,
    "Dress":3,
    "Coat":4,
    "Sandal":5,
    "Shirt":6,
    "Sneaker":7,
    "Bag":8,
    "Ankle_boot":9
}

In [6]:
# 원하는 Label number 리스트로 담기
is_target = [False for _ in range(10)]
target_labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle_boot"] #증강할 옷 종류 담기
for label in target_labels:
    is_target[label_dict[label]] = True
target_res = [[] for _ in range(10)]

train_count = X_train.shape[0]
for ind in range(train_count):
    if is_target[X_label[ind]]:
        target_res[X_label[ind]].append(ind)

In [7]:
origin_train = X_train.values # pandas.DF -> numpy.ARRAY
origin_train = origin_train.reshape(-1, 28, 28, 1) # 60000x28x28로 변환

In [8]:
augmented_train = train.copy()
# plt.imshow(origin_train[59999], cmap='gray')
# plt.axis('off')
# plt.show()

print(f"Before data shape: {origin_train.shape}")
image_generator = ImageDataGenerator(
            width_shift_range=0.01, # 세로 이동 범위
            height_shift_range=0.05,# 가로 이동 범위
            dtype="int64",
            fill_mode="constant",
            cval=0,
            )


Before data shape: (60000, 28, 28, 1)


In [9]:
sample_size = 300 # 각 label마다 추출할 표본 개수
augment_size = 20 # 각 표본별 증강할 개수

print(f"Estimated Augmented shape: ({(sample_size*len(target_labels)*augment_size) + origin_train.shape[0]}, 785)")

for label in target_labels:
    abstract_class  = random.sample(target_res[label_dict[label]], sample_size) #옷을 종류별로 sample_size만큼 무작위 선택
    labels = [label_dict[label] for _ in range(augment_size)]
    for ind in abstract_class:
        images = np.array([origin_train[ind] for _ in range(augment_size)])
        aug_imgs, aug_labels = image_generator.flow(images, labels, batch_size=augment_size, shuffle=False, seed=1127).next() #save_prefix='augmented', save_to_dir="./augment", 
        aug_imgs = aug_imgs.reshape(augment_size, 784)
        aug_imgs = np.insert(aug_imgs, 0, aug_labels, axis=1)
        df_augmented = pd.DataFrame(aug_imgs, columns=augmented_train.columns)
        augmented_train = pd.concat([augmented_train, df_augmented], ignore_index=True)

augmented_train = augmented_train.fillna(0.0)
print(f"After data shape: {augmented_train.shape}")
augmented_train = utils.shuffle(augmented_train)

Estimated Augmented shape: (120000, 785)
After data shape: (120000, 785)


In [10]:
augmented_train.to_csv('./120000_augmented.csv', index=False)

In [3]:

train = pd.read_csv('120000_augmented.csv')
train.isnull().any().sum()

0

In [4]:
testset = []
dir_name = '../public_test_dataset/data/'
lists = sorted(os.listdir(dir_name))
for idx, img in enumerate(lists):
    image_gray = cv2.imread(dir_name+img, cv2.IMREAD_GRAYSCALE)
    image = np.array(image_gray.reshape((784,)))
    # image = np.array(Image.open(dir_name+img)).reshape((784,))
    image = image.tolist()
    testset.append(image)
testset = np.array(testset)
test = pd.DataFrame(testset)
test = test/255.0

In [5]:
private_set = []
dir_name = '../private_test_dataset/data/'
lists = sorted(os.listdir(dir_name))
for idx, img in enumerate(lists):
    image_gray = cv2.imread(dir_name+img, cv2.IMREAD_GRAYSCALE)
    image = np.array(image_gray.reshape((784,)))
    # image = np.array(Image.open(dir_name+img)).reshape((784,))
    image = image.tolist()
    private_set.append(image)
private_set = np.array(private_set)
private = pd.DataFrame(private_set)
private = private/255.0

In [6]:
train.shape

(120000, 785)

In [7]:
test.shape

(10000, 784)

In [8]:
private.shape

(15000, 784)

In [9]:
df_train = train.copy()
df_test = test.copy()
df_private = private.copy()

In [10]:
X_train= df_train.drop(['label'],axis = 1)
y_train = df_train['label']
X_test = df_test
X_private = df_private

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_private = X_private.astype('float32')
X_train /= 255.0

In [11]:
# X_train과 X_label을 하나의 데이터 프레임으로 합침
df_train = pd.concat([X_train, y_train], axis=1)

# 데이터 프레임을 섞음
df_train = df_train.sample(frac=1, random_state=42)

# 섞인 데이터 프레임에서 훈련 데이터와 레이블을 다시 분리
X_train = df_train.drop(['label'], axis=1)
y_train = df_train['label']

In [12]:
def apply_hog(images):
    result = []
    for image in images:
        hog_features = hog(image, orientations=6, pixels_per_cell=(3, 3),
                        cells_per_block=(2, 2), block_norm='L2')
        result.append(hog_features)
    return result


In [13]:
print(X_train.shape)
print(X_test.shape)
print(X_private.shape)

(120000, 784)
(10000, 784)
(15000, 784)


In [14]:
X_train = np.array(X_train)
X_train = X_train.reshape(-1, 28, 28)
X_test = np.array(X_test)
X_test = X_test.reshape(-1, 28, 28)
X_private = np.array(X_private)
X_private = X_private.reshape(-1, 28, 28)

In [15]:
print(X_train.shape)
print(X_test.shape)
print(X_private.shape)

(120000, 28, 28)
(10000, 28, 28)
(15000, 28, 28)


In [16]:
X_train_hog = apply_hog(X_train)
X_test_hog = apply_hog(X_test)
X_private_hog = apply_hog(X_private)
X_train_hog = np.array(X_train_hog)
X_test_hog = np.array(X_test_hog)
X_private_hog = np.array(X_private_hog)


In [17]:
print("HOG 특성을 적용한 X_train의 형태: ", np.array(X_train_hog).shape)
print("HOG 특성을 적용한 X_test의 형태: ", np.array(X_test_hog).shape)
print("HOG 특성을 적용한 X_private의 형태: ", np.array(X_private_hog).shape)

HOG 특성을 적용한 X_train의 형태:  (120000, 1536)
HOG 특성을 적용한 X_test의 형태:  (10000, 1536)
HOG 특성을 적용한 X_private의 형태:  (15000, 1536)


In [18]:
pca = PCA(n_components=466, random_state=45)
pca.fit(X_train_hog)
X_train_pca = pca.transform(X_train_hog)
X_test_pca = pca.transform(X_test_hog)
X_private_pca = pca.transform(X_private_hog)

X_train_PCA1 = pd.DataFrame(X_train_pca)
X_test_PCA1 = pd.DataFrame(X_test_pca)
X_private_PCA1 = pd.DataFrame(X_private_pca)

In [19]:
optimal_c = 8
svc = SVC(gamma='scale',kernel='rbf',C=optimal_c, random_state=45)
svc.fit(X_train_PCA1,y_train)

SVC(C=8, random_state=45)

In [20]:
svc_pred = svc.predict(X_test_PCA1)
svc_private = svc.predict(X_private_PCA1)

In [21]:
f= open("testResult_public.txt","w+")
for idx, y in enumerate(svc_pred):
    num_str = str(idx).zfill(5)
    f.write(num_str + " " + str(int(y)) + "\n")
f.close()

In [22]:
f_private= open("testResult_private.txt","w+")
for idx_private, y_private in enumerate(svc_private):
    img_num_private = str(idx_private).zfill(5)
    f_private.write(img_num_private + " " + str(int(y_private)) + "\n")
f_private.close()

In [23]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

testResult_path = 'testResult_public.txt'
label_path = '../mAP/label.txt'

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()
    

# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []
    
    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1
        
        # preciison, recall 계산            
        if TP+FN != 0: 
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    AP.append(auc(temp_recall, temp_precision))

mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.2f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.2f} |\n".format("mAP", mAP)
test_mAP = mAP
print("Test mAP (public test) : {}".format(test_mAP))
print(table)


Test mAP (public test) : 0.8597482817130709
| Class         | AP            |
|---------------|---------------|
| T-shirt/top   | 0.79          |
| Trouser       | 0.98          |
| Pullover      | 0.79          |
| Dress         | 0.83          |
| Coat          | 0.77          |
| Sandal        | 0.98          |
| Shirt         | 0.64          |
| Sneaker       | 0.92          |
| Bag           | 0.97          |
| Ankle boot    | 0.93          |
| mAP           | 0.86          |

